## Feature Enginnering
1. Dividing dataset into modeling part and out-of-time part
2. Constructing variables separately on two datasets

In [59]:
import pandas as pd
import numpy as np
import time

In [37]:
# Read in cleaned dataset
data = pd.read_csv('data_0416.csv')

In [38]:
# Remove index columns and unify date format
data = data.drop(data.columns[0:2], axis = 1)
data['DATE'] = pd.to_datetime(data['DATE']).dt.date
data['RELEASE_DATE'] = pd.to_datetime(data['RELEASE_DATE']).dt.date

In [139]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2226075 entries, 1804633 to 1609613
Data columns (total 15 columns):
Unnamed: 0                                     int64
Unnamed: 0.1                                   int64
HASHED_ATOM_CUSTOMER_ID                        object
OS_TYPE_e                                      object
PRODUCTION_ID                                  int64
RATINGS                                        int64
MOVIE_DETAILS_PAGES_VIEWED_BY_CUSTOMER         int64
MOVIE_DETAILS_PAGES_VIEWED_BY_ALL_CUSTOMERS    int64
USER_SHARE_OF_ALL_MOVIE_DETAIL_PAGES           float64
WANT_TO_SEE_CLICKS                             int64
TRAILER_VIEWS                                  int64
LOAD_TIME_e                                    object
TICKETS                                        float64
RELEASE_DATE                                   object
DATE                                           datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(8), object(4)
memory usa

## Dividing Dataset

In [50]:
# subset an out-of-time dataset for future testing
data = data.sort_values(by = ['DATE'])
print(data['DATE'].min(), data['DATE'].max())

2017-04-01 2019-01-08


In [51]:
# Check the date at 80% of the dataset
data.iloc[[int(len(data) * 0.8)]]['DATE'] #2018-06-22

243679    2018-06-22
Name: DATE, dtype: object

In [68]:
# All rows after 2018-06-22, excluding 2018-06-22, will be subset into out-of-time dataset
print("oot dataset covers", 
      data[data.DATE > pd.to_datetime('2018-06-22').date()].shape[0] / len(data) * 100, 
      "% of the entire datset")
oot_data = data[data.DATE > pd.to_datetime('2018-06-22').date()]
print("the rest of the dataset covers", 
      data[data.DATE <= pd.to_datetime('2018-06-22').date()].shape[0] / len(data) * 100, 
      "% of the entire datset")
data_for_model = data[data.DATE <= pd.to_datetime('2018-06-22').date()]

oot dataset covers 19.99209370753456 % of the entire datset
oot dataset covers 80.00790629246544 % of the entire datset


## Variable Construction

In [120]:
# Prepare data for rolling window calculation
data_for_model['DATE'] = pd.to_datetime(data_for_model['DATE'])

In [74]:
#Pages-window
start_time = time.time()
data_for_model = data_for_model.sort_values(by = ['HASHED_ATOM_CUSTOMER_ID', 'DATE'])
data_index = data_for_model.set_index('DATE')
for agg in ['mean', 'max', 'median', 'sum', 'count']: 
    for days in ['1d', '3d', '7d', '14d', '30d']:
        data_for_model[agg + '_' + 'HASHED_ATOM_CUSTOMER_ID' + "_" + days+'_pages'] = getattr(data_index.groupby('HASHED_ATOM_CUSTOMER_ID')['MOVIE_DETAILS_PAGES_VIEWED_BY_CUSTOMER'].rolling(days),agg)().values
       # data['Actual/' + agg + "_" + 'HASHED_ATOM_CUSTOMER_ID' + "_" + days] = data['MOVIE_DETAILS_PAGES_VIEWED_BY_CUSTOMER']/data[agg + '_' + 'HASHED_ATOM_CUSTOMER_ID' + "_" + days]
print("--- %s seconds ---" % (time.time() - start_time))

--- 18278.720068216324 seconds ---


In [122]:
# Pages-window (extend window)
start_time = time.time()
data_for_model = data_for_model.sort_values(by = ['HASHED_ATOM_CUSTOMER_ID', 'DATE'])
data_index = data_for_model.set_index('DATE')
for agg in ['mean', 'max', 'median', 'sum', 'count']: 
    for days in ['28d', '35d', '42d', '49d', '56d']:
        data_for_model[agg + '_' + 'HASHED_ATOM_CUSTOMER_ID' + "_" + days + '_pages'] = getattr(data_index.groupby('HASHED_ATOM_CUSTOMER_ID')['MOVIE_DETAILS_PAGES_VIEWED_BY_CUSTOMER'].rolling(days),agg)().values
       # data['Actual/' + agg + "_" + 'HASHED_ATOM_CUSTOMER_ID' + "_" + days] = data['MOVIE_DETAILS_PAGES_VIEWED_BY_CUSTOMER']/data[agg + '_' + 'HASHED_ATOM_CUSTOMER_ID' + "_" + days]
print("--- %s seconds ---" % (time.time() - start_time))

--- 15435.411114931107 seconds ---


In [131]:
# RATINGS-window
start_time = time.time()
data_for_model = data_for_model.sort_values(by = ['HASHED_ATOM_CUSTOMER_ID', 'DATE'])
data_index = data_for_model.set_index('DATE')
for agg in ['mean', 'max', 'median', 'sum', 'count']: 
    for days in ['1d', '3d', '7d', '14d', '28d', '30d', '35d', '42d', '49d', '56d']:
        data_for_model[agg + '_' + 'HASHED_ATOM_CUSTOMER_ID' + "_" + days + '_ratings'] = getattr(data_index.groupby('HASHED_ATOM_CUSTOMER_ID')['RATINGS'].rolling(days),agg)().values
        #data['Actual/' + agg + "_" + 'HASHED_ATOM_CUSTOMER_ID' + "_" + days] = data['RATINGS']/data[agg + '_' + 'HASHED_ATOM_CUSTOMER_ID' + "_" + days]
print("--- %s seconds ---" % (time.time() - start_time))

--- 31028.102921009064 seconds ---


In [79]:
# drop unnecessary column
data_for_model = data_for_model.drop(columns = ['LOAD_TIME_e'])

In [86]:
# Variable DATE_TO_RELEASE
data_for_model['RELEASE_DATE'] = pd.to_datetime(data_for_model['RELEASE_DATE']).dt.date
data_for_model['DATE'] = pd.to_datetime(data_for_model['DATE']).dt.date
data_for_model['DATE_TO_RELEASE'] = data_for_model['DATE'] - data_for_model['RELEASE_DATE']

In [87]:
# sort dataset by date
data_for_model = data_for_model.sort_values(by = 'DATE')

In [88]:
# Variable CNT_ENGAGE
data_for_model['CNT_ENGAGE'] = data_for_model.groupby('HASHED_ATOM_CUSTOMER_ID').cumcount()

In [91]:
data_for_model['CNT_ENGAGE'].describe()

count    1.781036e+06
mean     1.088320e+02
std      1.173593e+03
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      1.000000e+00
max      1.950300e+04
Name: CNT_ENGAGE, dtype: float64

In [92]:
# Variable CNT_WANT_TO_SEE_CLICKS
data_for_model['CNT_WANT_TO_SEE_CLICKS'] = data_for_model.groupby('PRODUCTION_ID')['WANT_TO_SEE_CLICKS'].cumsum()

In [93]:
# Variable: CNT_TRAILER_VIEWS
data_for_model['CNT_TRAILER_VIEWS'] = data_for_model.groupby('PRODUCTION_ID')['TRAILER_VIEWS'].cumsum()

In [94]:
# Variable CNT_MOVIE_ENGAGEMENT
data_for_model['CNT_MOVIE_ENGAGEMENT'] = data_for_model.groupby('HASHED_ATOM_CUSTOMER_ID')['PRODUCTION_ID'].cumcount()

In [95]:
data_for_model['CNT_MOVIE_ENGAGEMENT'].describe()

count    1.781036e+06
mean     1.088320e+02
std      1.173593e+03
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      1.000000e+00
max      1.950300e+04
Name: CNT_MOVIE_ENGAGEMENT, dtype: float64

In [96]:
# Variable TICKETS_CLASS
data_for_model['TICKETS_CLASS'] = data_for_model['TICKETS'].apply(lambda x: 3 if x >= 3 else x)

In [98]:
# Variable WEEKDAY_OF_ENGAGEMENT
data_for_model['WEEKDAY_OF_ENGAGEMENT'] = data_for_model['DATE'].apply(lambda x: x.weekday())

In [143]:
data_for_model.info(verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1781036 entries, 1804633 to 1609613
Data columns (total 119 columns):
HASHED_ATOM_CUSTOMER_ID                        1781036 non-null object
OS_TYPE_e                                      1781036 non-null object
PRODUCTION_ID                                  1781036 non-null int64
RATINGS                                        1781036 non-null int64
MOVIE_DETAILS_PAGES_VIEWED_BY_CUSTOMER         1781036 non-null int64
MOVIE_DETAILS_PAGES_VIEWED_BY_ALL_CUSTOMERS    1781036 non-null int64
USER_SHARE_OF_ALL_MOVIE_DETAIL_PAGES           1781036 non-null float64
WANT_TO_SEE_CLICKS                             1781036 non-null int64
TRAILER_VIEWS                                  1781036 non-null int64
TICKETS                                        1781036 non-null float64
RELEASE_DATE                                   1781036 non-null object
DATE                                           1781036 non-null datetime64[ns]
mean_HASHED_ATOM_CUSTOME

### Repeat the same variable construction process for OOT dataset

In [133]:
# Prepare OOT data for rolling window calculation
oot_data['DATE'] = pd.to_datetime(oot_data['DATE'])

In [77]:
#Pages-window (OOT)
start_time = time.time()
oot_data = oot_data.sort_values(by = ['HASHED_ATOM_CUSTOMER_ID', 'DATE'])
data_index = oot_data.set_index('DATE')
for agg in ['mean', 'max', 'median', 'sum', 'count']: 
    for days in ['1d', '3d', '7d', '14d', '30d']:
        oot_data[agg + '_' + 'HASHED_ATOM_CUSTOMER_ID' + "_" + days+'_pages'] = getattr(data_index.groupby('HASHED_ATOM_CUSTOMER_ID')['MOVIE_DETAILS_PAGES_VIEWED_BY_CUSTOMER'].rolling(days),agg)().values
       # data['Actual/' + agg + "_" + 'HASHED_ATOM_CUSTOMER_ID' + "_" + days] = data['MOVIE_DETAILS_PAGES_VIEWED_BY_CUSTOMER']/data[agg + '_' + 'HASHED_ATOM_CUSTOMER_ID' + "_" + days]
print("--- %s seconds ---" % (time.time() - start_time))

--- 2812.2667944431305 seconds ---


In [134]:
#Pages-window (OOT) extend
start_time = time.time()
oot_data = oot_data.sort_values(by = ['HASHED_ATOM_CUSTOMER_ID', 'DATE'])
data_index = oot_data.set_index('DATE')
for agg in ['mean', 'max', 'median', 'sum', 'count']: 
    for days in ['28d', '35d', '42d', '49d', '56d']:
        oot_data[agg + '_' + 'HASHED_ATOM_CUSTOMER_ID' + "_" + days+'_pages'] = getattr(data_index.groupby('HASHED_ATOM_CUSTOMER_ID')['MOVIE_DETAILS_PAGES_VIEWED_BY_CUSTOMER'].rolling(days),agg)().values
       # data['Actual/' + agg + "_" + 'HASHED_ATOM_CUSTOMER_ID' + "_" + days] = data['MOVIE_DETAILS_PAGES_VIEWED_BY_CUSTOMER']/data[agg + '_' + 'HASHED_ATOM_CUSTOMER_ID' + "_" + days]
print("--- %s seconds ---" % (time.time() - start_time))

--- 2826.1970880031586 seconds ---


In [135]:
#RATINGS-window (OOT)
start_time = time.time()
oot_data = oot_data.sort_values(by = ['HASHED_ATOM_CUSTOMER_ID', 'DATE'])
data_index = oot_data.set_index('DATE')
for agg in ['mean', 'max', 'median', 'sum', 'count']: 
    for days in ['1d', '3d', '7d', '14d', '28d', '30d', '35d', '42d', '49d', '56d']:
        oot_data[agg + '_' + 'HASHED_ATOM_CUSTOMER_ID' + "_" + days + '_ratings'] = getattr(data_index.groupby('HASHED_ATOM_CUSTOMER_ID')['RATINGS'].rolling(days),agg)().values
        #data['Actual/' + agg + "_" + 'HASHED_ATOM_CUSTOMER_ID' + "_" + days] = data['RATINGS']/data[agg + '_' + 'HASHED_ATOM_CUSTOMER_ID' + "_" + days]
print("--- %s seconds ---" % (time.time() - start_time))

--- 5614.406920194626 seconds ---


In [99]:
oot_data = oot_data.drop(columns = ['LOAD_TIME_e'])

# Variable DATE_TO_RELEASE
oot_data['RELEASE_DATE'] = pd.to_datetime(oot_data['RELEASE_DATE']).dt.date
oot_data['DATE'] = pd.to_datetime(oot_data['DATE']).dt.date
oot_data['DATE_TO_RELEASE'] = oot_data['DATE'] - oot_data['RELEASE_DATE']

oot_data = oot_data.sort_values(by = 'DATE')

# Variable CNT_ENGAGE
oot_data['CNT_ENGAGE'] = oot_data.groupby('HASHED_ATOM_CUSTOMER_ID').cumcount()

oot_data['CNT_ENGAGE'].describe()

count    445039.000000
mean        391.931379
std        2166.873331
min           0.000000
25%           0.000000
50%           0.000000
75%           2.000000
max       18637.000000
Name: CNT_ENGAGE, dtype: float64

In [100]:
# Variable CNT_WANT_TO_SEE_CLICKS
oot_data['CNT_WANT_TO_SEE_CLICKS'] = oot_data.groupby('PRODUCTION_ID')['WANT_TO_SEE_CLICKS'].cumsum()

# Variable: CNT_TRAILER_VIEWS
oot_data['CNT_TRAILER_VIEWS'] = oot_data.groupby('PRODUCTION_ID')['TRAILER_VIEWS'].cumsum()

# Variable CNT_MOVIE_ENGAGEMENT
oot_data['CNT_MOVIE_ENGAGEMENT'] = oot_data.groupby('HASHED_ATOM_CUSTOMER_ID')['PRODUCTION_ID'].cumcount()

oot_data['CNT_MOVIE_ENGAGEMENT'].describe()

count    445039.000000
mean        391.931379
std        2166.873331
min           0.000000
25%           0.000000
50%           0.000000
75%           2.000000
max       18637.000000
Name: CNT_MOVIE_ENGAGEMENT, dtype: float64

In [101]:
# Variable TICKETS_CLASS
oot_data['TICKETS_CLASS'] = oot_data['TICKETS'].apply(lambda x: 3 if x >= 3 else x)
oot_data['TICKETS_CLASS'].describe()

count    445039.000000
mean          0.367871
std           0.693109
min           0.000000
25%           0.000000
50%           0.000000
75%           1.000000
max           3.000000
Name: TICKETS_CLASS, dtype: float64

In [144]:
oot_data['TICKETS_CLASS'].unique()

array([0., 1., 2., 3.])

In [145]:
# Variable WEEKDAY_OF_ENGAGEMENT
oot_data['WEEKDAY_OF_ENGAGEMENT'] = oot_data['DATE'].apply(lambda x: x.weekday())

In [142]:
oot_data.info(verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 445039 entries, 266724 to 278173
Data columns (total 119 columns):
HASHED_ATOM_CUSTOMER_ID                        445039 non-null object
OS_TYPE_e                                      445039 non-null object
PRODUCTION_ID                                  445039 non-null int64
RATINGS                                        445039 non-null int64
MOVIE_DETAILS_PAGES_VIEWED_BY_CUSTOMER         445039 non-null int64
MOVIE_DETAILS_PAGES_VIEWED_BY_ALL_CUSTOMERS    445039 non-null int64
USER_SHARE_OF_ALL_MOVIE_DETAIL_PAGES           445039 non-null float64
WANT_TO_SEE_CLICKS                             445039 non-null int64
TRAILER_VIEWS                                  445039 non-null int64
TICKETS                                        445039 non-null float64
RELEASE_DATE                                   445039 non-null object
DATE                                           445039 non-null datetime64[ns]
mean_HASHED_ATOM_CUSTOMER_ID_1d_pages  

In [22]:
data[data['DATE']=='2017-04-01']['MOVIE_DETAILS_PAGES_VIEWED_BY_CUSTOMER'].sum()

14860

In [23]:
data[data['DATE']=='2017-04-01']['MOVIE_DETAILS_PAGES_VIEWED_BY_ALL_CUSTOMERS'].unique()

array([223920])

In [138]:
print(data_for_model.shape, oot_data.shape)

(1781036, 119) (445039, 119)


In [146]:
# Exporting the datasets
data_for_model.to_csv('modeling_data_v2.csv', index = False)

In [147]:
oot_data.to_csv('oot_data_v2.csv', index = False)